### Using COLMAP Structure From Motion (SFM) Package

see <https://github.com/Fyusion/LLFF/tree/master> for more documentation

In [ ]:
from pillow_heif import register_heif_opener
from matplotlib import pyplot as plt
import numpy as np
import os
from colmapParsingUtils import *
from scipy.spatial.transform import Rotation as R
import cv2

import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
from PIL import Image

# #limit GPU memory ------------------------------------------------
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# if gpus:
#   try:
#     memlim = 12*1024
#     tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
#   except RuntimeError as e:
#     print(e)
# #-----------------------------------------------------------------


In [ ]:
# import os
# import sys
# current = os.getcwd()
# sys.path.append(current+"/LLFF")

# from LLFF.llff.poses.pose_utils import gen_poses
# import sys

#<imgs2poses.py>:
# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument('--match_type', type=str, 
#                     default='exhaustive_matcher', help='type of matcher used.  Valid options: \
#                     exhaustive_matcher sequential_matcher.  Other matchers not supported at this time')
# parser.add_argument('scenedir', type=str,
#                     help='input scene directory')
# args = parser.parse_args()

# if args.match_type != 'exhaustive_matcher' and args.match_type != 'sequential_matcher':
#     print('ERROR: matcher type ' + args.match_type + ' is not valid.  Aborting')
#     sys.exit()

# if __name__=='__main__':
#     gen_poses(args.scenedir, args.match_type)

In [ ]:
#convert HEIC images to PNG

# register_heif_opener()

# # image_i = Image.open('desk_images/IMG_2072.HEIC')
# image_i = Image.open('bike_images/IMG_2167.HEIC')
# i = np.asarray(image_i)
# i = i[516:-516,12:-12,:] #crop square
# i = i[::3,::3,:]
# print(np.shape(i))
# plt.imshow(i)


In [ ]:
#loop through subdirectory and convert each .HEIC to similarly named .PNG
register_heif_opener()
# rootdir = 'desk_images'
rootdir = 'bike_images'

mega_image_array = np.zeros([0,1000,1000,3])

for subdir, dirs, files in os.walk(rootdir):
    for file in sorted(files):
        if file.endswith(("HEIC")):
            print(file)
            image_i = Image.open(rootdir+'/'+file)
            i = np.asarray(image_i)
            #crop square
            if np.shape(i)[0] > np.shape(i)[1]:
                i = i[516:-516,12:-12,:]
            else:
                i = i[12:-12,516:-516,:]
            i = i[::3,::3,:]
            mega_image_array = np.append(mega_image_array, i[None,:,:,:], axis = 0)
            im = Image.fromarray(i)
#             im.save(rootdir+"/"+file[:-5]+'.PNG')
#             os.remove(file) #remove HEIC image


In [ ]:
# print(np.shape(mega_image_array[0]))
    
# for n in range(10):
#     plt.imshow(mega_image_array[n]/255)

In [ ]:
# np.savez_compressed("desk_images", mega_image_array)

In [ ]:
#load poses estimated by COLMAP

images_from_colmap = read_images_text("sparseBike/0/images.txt")
cameras = read_cameras_text("sparseBike/0/cameras.txt")
pts3d = read_points3D_text("sparseBike/0/points3D.txt")
print(images_from_colmap[1])
# print(pts3d[1])
print("\n",cameras[10])

In [ ]:
#convert COLMAP poses (xyz,quats) to rotm

poses = np.zeros([len(images_from_colmap),4,4])
# focal = 

# images = np.zeros([len(poses),100,100,3])
images = np.zeros([len(poses),250,250,3])

#loop through <images_from_colmap> to get 3D poses of cameras at each timestamp
print(len(images_from_colmap))
for n in range(1,len(images_from_colmap)+1):
#     print(images[n].qvec)
    rot33 = R.from_quat(images_from_colmap[n].qvec).as_matrix()
    trans31 = images_from_colmap[n].tvec[:,None]
    poses[n-1] = np.append(np.append(rot33, trans31, axis = 1), np.array([[0,0,0,1.]]), axis = 0)

    #sync order of images with order of poses
    temp = cv2.imread("bike_images/"+images_from_colmap[n].name)/255    
#     print(np.shape(temp))
#     print(np.shape(images[n-1]))
#     images[n-1] = temp[::10,::10,:]#downsample
    images[n-1] = temp[::4,::4,:]#downsample

#GET REST OF PARAMS NEEDED FOR tinyNeRF format~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
#out of order??
# images = mega_image_array/255 #rescale mega_image_array to [0,1]
# images = images[:,::10,::10,:]#downsample

H,W = images.shape[1:3]
testimg, testpose = images[55], poses[55]

# focal = cameras[1].params[0] #test- see if same focal length can be shared across all images
# print(focal)
focal = np.array(984.411).astype(np.double)
print(type(focal))
print(focal)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
#verify images are set up correclty
plt.imshow(images[0])
# plt.imshow(images_from_colmap[1])

# Construct NeRF

In [ ]:
def posenc(x):
  rets = [x]
  for i in range(L_embed):
    for fn in [tf.sin, tf.cos]:
      rets.append(fn(2.**i * x))
  return tf.concat(rets, -1)

L_embed = 6
embed_fn = posenc
# L_embed = 0
# embed_fn = tf.identity

def init_model(D=8, W=256):
    relu = tf.keras.layers.ReLU()    
    dense = lambda W=W, act=relu : tf.keras.layers.Dense(W, activation=act)

    inputs = tf.keras.Input(shape=(3 + 3*2*L_embed)) 
    outputs = inputs
    for i in range(D):
        outputs = dense()(outputs)
        if i%4==0 and i>0:
            outputs = tf.concat([outputs, inputs], -1)
    outputs = dense(4, act=None)(outputs)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_rays(H, W, focal, c2w):
    i, j = tf.meshgrid(tf.range(W, dtype=tf.float32), tf.range(H, dtype=tf.float32), indexing='xy')
    dirs = tf.stack([(i-W*.5)/focal, -(j-H*.5)/focal, -tf.ones_like(i)], -1)
    rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)
    rays_o = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d))
    return rays_o, rays_d



def render_rays(network_fn, rays_o, rays_d, near, far, N_samples, rand=False):

    def batchify(fn, chunk=1024*32):
        return lambda inputs : tf.concat([fn(inputs[i:i+chunk]) for i in range(0, inputs.shape[0], chunk)], 0)
    
    # Compute 3D query points
    z_vals = tf.linspace(near, far, N_samples) 
    if rand:
      z_vals += tf.random.uniform(list(rays_o.shape[:-1]) + [N_samples]) * (far-near)/N_samples
    pts = rays_o[...,None,:] + rays_d[...,None,:] * z_vals[...,:,None]
    
    # Run network
    pts_flat = tf.reshape(pts, [-1,3])
    pts_flat = embed_fn(pts_flat)
    raw = batchify(network_fn)(pts_flat)
    raw = tf.reshape(raw, list(pts.shape[:-1]) + [4])
    
    # Compute opacities and colors
    sigma_a = tf.nn.relu(raw[...,3])
    rgb = tf.math.sigmoid(raw[...,:3]) 
    
    # Do volume rendering
    dists = tf.concat([z_vals[..., 1:] - z_vals[..., :-1], tf.broadcast_to([1e10], z_vals[...,:1].shape)], -1) 
    alpha = 1.-tf.exp(-sigma_a * dists)  
    weights = alpha * tf.math.cumprod(1.-alpha + 1e-10, -1, exclusive=True)
    
    rgb_map = tf.reduce_sum(weights[...,None] * rgb, -2) 
    depth_map = tf.reduce_sum(weights * z_vals, -1) 
    acc_map = tf.reduce_sum(weights, -1)

    return rgb_map, depth_map, acc_map

In [ ]:
model = init_model(D=8, W=256)
# model = init_model(D=8, W=64) #test simple model -- nope
optimizer = tf.keras.optimizers.Adam(5e-4)

N_samples = 32 #64 #decrease as needed for VRAM(?)
N_iters = 1000
psnrs = []
iternums = []
i_plot = 10 #25

import time
t = time.time()
for i in range(N_iters+1):
    
    img_i = np.random.randint(images.shape[0])
    target = images[img_i]
    pose = poses[img_i]
    
    rays_o, rays_d = get_rays(H, W, focal, pose)
    rays_o = tf.cast(rays_o, tf.float32)
    rays_d = tf.cast(rays_d, tf.float32)
    
    with tf.GradientTape() as tape:
        rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples, rand=True)
        loss = tf.reduce_mean(tf.square(rgb - target))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    if i%i_plot==0:
        print(i, (time.time() - t) / i_plot, 'secs per iter')
        t = time.time()
        
        # Render the holdout view for logging
        rays_o, rays_d = get_rays(H, W, focal, testpose)
        rays_o = tf.cast(rays_o, tf.float32)
        rays_d = tf.cast(rays_d, tf.float32)
        
        rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples)
        loss = tf.reduce_mean(tf.square(rgb - testimg))
        psnr = -10. * tf.math.log(loss) / tf.math.log(10.)

        psnrs.append(psnr.numpy())
        iternums.append(i)
        
        plt.figure(figsize=(10,4))
        plt.subplot(121)
        plt.imshow(rgb)
        plt.title(f'Iteration: {i}')
        plt.subplot(122)
        plt.plot(iternums, psnrs)
        plt.title('PSNR')
        plt.show()

print('Done')

In [ ]:
%matplotlib inline
from ipywidgets import interactive, widgets


trans_t = lambda t : tf.convert_to_tensor([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,t],
    [0,0,0,1],
], dtype=tf.float32)

rot_phi = lambda phi : tf.convert_to_tensor([
    [1,0,0,0],
    [0,tf.cos(phi),-tf.sin(phi),0],
    [0,tf.sin(phi), tf.cos(phi),0],
    [0,0,0,1],
], dtype=tf.float32)

rot_theta = lambda th : tf.convert_to_tensor([
    [tf.cos(th),0,-tf.sin(th),0],
    [0,1,0,0],
    [tf.sin(th),0, tf.cos(th),0],
    [0,0,0,1],
], dtype=tf.float32)


def pose_spherical(theta, phi, radius):
    c2w = trans_t(radius)
    c2w = rot_phi(phi/180.*np.pi) @ c2w
    c2w = rot_theta(theta/180.*np.pi) @ c2w
    c2w = np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) @ c2w
    return c2w


def f(**kwargs):
    c2w = pose_spherical(**kwargs)
    rays_o, rays_d = get_rays(H, W, focal, c2w[:3,:4])
    rgb, depth, acc = render_rays(model, rays_o, rays_d, near=2., far=6., N_samples=N_samples)
    img = np.clip(rgb,0,1)
    
    plt.figure(2, figsize=(20,6))
    plt.imshow(img)
    plt.show()
    

sldr = lambda v, mi, ma: widgets.FloatSlider(
    value=v,
    min=mi,
    max=ma,
    step=.01,
)

names = [
    ['theta', [100., 0., 360]],
    ['phi', [-30., -90, 0]],
    ['radius', [4., 3., 5.]],
]

interactive_plot = interactive(f, **{s[0] : sldr(*s[1]) for s in names})
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot